In [1]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle

In [12]:
path = "../myData"

In [13]:
os.listdir(path)

['9', '0', '7', '6', '1', '8', '4', '3', '2', '5']

In [14]:
X = []
y = []

for x in range(0, 10):
    picList = os.listdir(path + "/" + str(x))
    for z in picList:
        curImg = cv2.imread(path + "/" + str(x) + "/" + z)
        curImg = cv2.resize(curImg, (32, 32))
        X.append(curImg)
        y.append(x)
    print(x)
        

0
1
2
3
4
5
6
7
8
9


In [15]:
X = np.array(X)
y = np.array(y)

In [17]:
for i in range(0, 10):
    print("digit " + str(i) + " - " + str(len(np.where(y == i)[0])))

digit 0 - 1016
digit 1 - 1016
digit 2 - 1016
digit 3 - 1016
digit 4 - 1016
digit 5 - 1016
digit 6 - 1016
digit 7 - 1016
digit 8 - 1016
digit 9 - 1016


In [18]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5)

In [20]:
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(6096, 32, 32, 3) (6096,) (2032, 32, 32, 3) (2032,) (2032, 32, 32, 3) (2032,)


In [22]:
def preProcessing(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.equalizeHist(img)
    img = img/255
    return img

In [23]:
X_train_preprocessed = np.array(list(map(preProcessing, X_train)))
X_val_preprocessed = np.array(list(map(preProcessing, X_val)))
X_test_preprocessed = np.array(list(map(preProcessing, X_test)))

In [25]:
X_train_reshaped = X_train_preprocessed.reshape(X_train_preprocessed.shape[0], X_train_preprocessed.shape[1], X_train_preprocessed.shape[2], 1)
X_val_reshaped = X_val_preprocessed.reshape(X_val_preprocessed.shape[0], X_val_preprocessed.shape[1], X_val_preprocessed.shape[2], 1)
X_test_reshaped = X_test_preprocessed.reshape(X_test_preprocessed.shape[0], X_test_preprocessed.shape[1], X_test_preprocessed.shape[2], 1)

In [26]:
data_gen = tf.keras.preprocessing.image.ImageDataGenerator(width_shift_range =0.1,
                                                          height_shift_range = 0.1,
                                                          zoom_range = 0.2,
                                                          shear_range = 0.1,
                                                          rotation_range = 10)

data_gen.fit(X_train_reshaped)

In [27]:
y_train_encoded = tf.keras.utils.to_categorical(y_train, 10)
y_val_encoded = tf.keras.utils.to_categorical(y_val, 10)
y_test_encoded = tf.keras.utils.to_categorical(y_test, 10)

In [35]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(60, (5,5), input_shape=(32, 32, 1), activation='relu'))
model.add(tf.keras.layers.Conv2D(60, (5,5), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(30, (3,3), activation='relu'))
model.add(tf.keras.layers.Conv2D(30, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(500, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

learning_rate = 0.001  # You can tune this value
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

/Users/hasangaranasinghe/Documents/Projects/Text-Detection/.venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 28, 28, 60)     │         1,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 24, 24, 60)     │        90,060 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 12, 12, 60)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 10, 10, 30)     │        16,230 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 8, 8, 30)       │         8,130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 4, 4, 30)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4, 4, 30)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 480)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 500)            │       240,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 500)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         5,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 361,490 (1.38 MB)

 Trainable params: 361,490 (1.38 MB)

 Non-trainable params: 0 (0.00 B)

In [36]:
history = model.fit(
    data_gen.flow(X_train_reshaped, y_train_encoded, batch_size=50),
    steps_per_epoch=2000,
    epochs=10,
    validation_data=(X_val_reshaped, y_val_encoded),
    shuffle=True
)

Epoch 1/10


/Users/hasangaranasinghe/Documents/Projects/Text-Detection/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 122/2000 ━━━━━━━━━━━━━━━━━━━━ 6:23 204ms/step - accuracy: 0.2860 - loss: 1.9252

2024-08-16 15:45:00.086635: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/opt/homebrew/Cellar/python@3.12/3.12.5/Frameworks/Python.framework/Versions/3.12/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


2000/2000 ━━━━━━━━━━━━━━━━━━━━ 31s 15ms/step - accuracy: 0.4797 - loss: 1.4586 - val_accuracy: 0.9434 - val_loss: 0.2187
Epoch 2/10
 122/2000 ━━━━━━━━━━━━━━━━━━━━ 6:23 204ms/step - accuracy: 0.8267 - loss: 0.5263

...

2000/2000 ━━━━━━━━━━━━━━━━━━━━ 28s 14ms/step - accuracy: 0.8477 - loss: 0.4679 - val_accuracy: 0.9651 - val_loss: 0.1333
Epoch 3/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 31s 15ms/step - accuracy: 0.8977 - loss: 0.3215 - val_accuracy: 0.9783 - val_loss: 0.0751
Epoch 4/10
 122/2000 ━━━━━━━━━━━━━━━━━━━━ 8:07 260ms/step - accuracy: 0.9192 - loss: 0.2561

2024-08-16 15:46:35.693203: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


2000/2000 ━━━━━━━━━━━━━━━━━━━━ 35s 17ms/step - accuracy: 0.9234 - loss: 0.2415 - val_accuracy: 0.9823 - val_loss: 0.0612
Epoch 5/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 29s 14ms/step - accuracy: 0.9344 - loss: 0.2064 - val_accuracy: 0.9877 - val_loss: 0.0355
Epoch 6/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 29s 14ms/step - accuracy: 0.9406 - loss: 0.1823 - val_accuracy: 0.9867 - val_loss: 0.0390
Epoch 7/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 28s 14ms/step - accuracy: 0.9526 - loss: 0.1498 - val_accuracy: 0.9872 - val_loss: 0.0347
Epoch 8/10
 122/2000 ━━━━━━━━━━━━━━━━━━━━ 7:15 232ms/step - accuracy: 0.9511 - loss: 0.1462

2024-08-16 15:48:32.791339: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


2000/2000 ━━━━━━━━━━━━━━━━━━━━ 32s 16ms/step - accuracy: 0.9514 - loss: 0.1474 - val_accuracy: 0.9892 - val_loss: 0.0329
Epoch 9/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 29s 14ms/step - accuracy: 0.9572 - loss: 0.1368 - val_accuracy: 0.9887 - val_loss: 0.0312
Epoch 10/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 30s 15ms/step - accuracy: 0.9649 - loss: 0.1178 - val_accuracy: 0.9921 - val_loss: 0.0245


In [37]:
score = model.evaluate(X_test_reshaped, y_test_encoded, verbose=0)
score

[0.01151807140558958, 0.9960629940032959]

In [38]:
pickle_out = open("../model_trained_X.p", "wb")
pickle.dump(model, pickle_out)
pickle_out.close()

In [46]:
img = cv2.imread("../image.png")
img = np.asarray(img)
img = cv2.resize(img, (32, 32))
img = preProcessing(img)
img = img.reshape(1, 32, 32, 1)
predictions = model.predict(img)
print("Prediction probabilities:", predictions)
classIndex = np.argmax(predictions, axis=1)
print(classIndex)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Prediction probabilities: [[1.2828022e-09 1.0167688e-11 6.7970831e-08 7.3135588e-08 4.2217973e-08
  5.1090354e-09 2.7702689e-09 7.2388758e-13 9.9999940e-01 3.1284117e-07]]
[8]
